In [5]:
from qiskit import *
from qiskit_aer import AerSimulator
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_experiments import *
from qiskit.quantum_info import state_fidelity

In [14]:
# Shadow tomography para o qubit recebido em um circuito de teleporte quântico
# Autor: ChatGPT (GPT-5 Thinking)
# Requisitos: qiskit, qiskit-aer, numpy, scipy (opcional: para sqrtm)

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np
from cmath import isclose

# --- Utilidades de álgebra ---
I2 = np.eye(2, dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)


def bloch_to_rho(rx, ry, rz):
    """Reconstrói rho = (I + r·σ)/2 a partir de (⟨X⟩,⟨Y⟩,⟨Z⟩)."""
    return 0.5*(I2 + rx*X + ry*Y + rz*Z)


def fidelity(rho, sigma):
    """Fidelidade de Uhlmann F(rho, sigma) = (Tr sqrt(sqrt(rho) sigma sqrt(rho)))^2"""
    # Implementação estável para 1 qubit via autovalores
    # (evita dependência do scipy)
    def sqrtm(A):
        w, V = np.linalg.eigh(A)
        w = np.clip(w, 0.0, None)
        return (V * np.sqrt(w)) @ V.conj().T
    S = sqrtm(rho)
    M = S @ sigma @ S
    w, _ = np.linalg.eigh((M + M.conj().T)/2)  # Hermitiza por segurança
    w = np.clip(w, 0.0, None)
    return float(np.real(np.sum(np.sqrt(w)))**2)

# --- Circuito de teleporte canônico (com correções condicionais reais) ---

def teleport_circuit(secret_prep=None):
    """
    Cria circuito de teleporte 3-qubits:
      q0: Secret (estado a ser teletransportado)
      q1: Alice
      q2: Bob (receptor final)
    Mede (q0,q1) e aplica correções condicionais verdadeiras em q2.

    Parâmetros
    ---------
    secret_prep : callable | None
        Função que recebe (qc, qubit_index) e prepara o estado secreto em q0.
        Por padrão, prepara |+> (H|0>). Ex.: lambda qc, q: qc.ry(theta, q); qc.rz(phi, q)
    """
    q = QuantumRegister(3, 'q')
    c0 = ClassicalRegister(1, 'c0')  # medição de q0
    c1 = ClassicalRegister(1, 'c1')  # medição de q1
    qc = QuantumCircuit(q, c0, c1, name='teleport')

    # Estado secreto |ψ> no q[0]
    if secret_prep is None:
        secret_prep = lambda qc_, qidx: qc_.h(qidx)  # |+>
    secret_prep(qc, q[0])

    # Par entre Alice (q1) e Bob (q2)
    qc.h(q[1])
    qc.cx(q[1], q[2])

    # Medição de Bell em (q0,q1)
    qc.cx(q[0], q[1])
    qc.h(q[0])
    qc.measure(q[0], c0)
    qc.measure(q[1], c1)

    # Correções condicionais em Bob (q2)
    # X se c1 == 1 ; Z se c0 == 1
    # Correções condicionais em Bob (q2)
    with qc.if_test((c1, 1)):
        qc.x(q[2])
    with qc.if_test((c0, 1)):
        qc.z(q[2])


    return qc

# --- Medições em bases de Pauli no qubit de Bob para "shadow"/tomografia ---

_basis_rot = {
    'Z': [],                 # medir Z: nada
    'X': [('h', )],          # medir X: H
    'Y': [('sdg',), ('h',)], # medir Y: S†; H
}


def _attach_pauli_measure_on_bob(base_circ: QuantumCircuit, basis: str):
    """Clona base_circ e adiciona rotação+medição do qubit de Bob (q2) em 'basis'."""
    circ = base_circ.copy()
    # Clássico extra para Bob
    cb = ClassicalRegister(1, f'cb_{basis}')
    circ.add_register(cb)
    # Rotações de base em q2
    for step in _basis_rot[basis]:
        gate = step[0]
        if gate == 'h':
            circ.h(2)
        elif gate == 'sdg':
            circ.sdg(2)
    circ.measure(2, cb)
    return circ


def estimate_pauli_expectations(sim, base_circ, shots_per_basis=2000, seed=1234):
    """
    Mede ⟨X⟩, ⟨Y⟩, ⟨Z⟩ do qubit de Bob (q2) executando o circuito de teleporte
    seguido de medições em bases de Pauli. Retorna dicionário com as médias.
    """
    exps = {}
    for basis in ['X','Y','Z']:
        circ = _attach_pauli_measure_on_bob(base_circ, basis)
        res = sim.run(circ, shots=shots_per_basis, seed_simulator=seed).result()
        counts = res.get_counts()
        # Extrair apenas o bit de Bob; get_counts retorna chaves tipo 'b c1 c0' dependendo da ordem
        # Estratégia: somar outcomes por paridade do bit mais significativo do último clássico adicionado.
        # Como cb foi adicionado por último, seu bit fica no início da string de bits retornada pelo Qiskit.
        p0 = sum(v for k,v in counts.items() if k[0]=='0')
        p1 = sum(v for k,v in counts.items() if k[0]=='1')
        total = p0 + p1 if (p0+p1)>0 else 1
        exps[basis] = (p0 - p1)/total
    return exps


def shadow_tomography_rho(secret_prep=None, shots_per_basis=2000, return_all=False, seed=1234):
    """
    Executa circuito de teleporte e reconstrói rho do qubit de Bob via
    medições em X, Y, Z (equivalente ao estimador de classical shadows
    especializado para 1 qubit).

    Parâmetros
    ----------
    secret_prep : callable | None
        Preparação do estado secreto em q0 (default: |+>).
    shots_per_basis : int
        Nº de shots por base (X, Y, Z).
    return_all : bool
        Se True, retorna também ⟨X⟩,⟨Y⟩,⟨Z⟩ e o circuito usado.
    seed : int
        Semente do simulador.

    Retorna
    -------
    rho_hat : np.ndarray (2x2 complex)
    (opcional) exps : dict
    (opcional) circ : QuantumCircuit
    """
    base = teleport_circuit(secret_prep=secret_prep)
    sim = AerSimulator()
    exps = estimate_pauli_expectations(sim, base, shots_per_basis=shots_per_basis, seed=seed)
    rho_hat = bloch_to_rho(exps['X'], exps['Y'], exps['Z'])
    if return_all:
        return rho_hat, exps, base
    return rho_hat

# --- Exemplo de uso: estado secreto |+> ---
if __name__ == '__main__':
    # |ψ> = |+>
    rho_hat, exps, circ = shadow_tomography_rho(return_all=True, shots_per_basis=4000)
    # Rho ideal de |+>
    ket_plus = (1/np.sqrt(2))*np.array([1,1], dtype=complex)
    rho_ideal = np.outer(ket_plus, ket_plus.conj())
    print('⟨X⟩,⟨Y⟩,⟨Z⟩ estimados:', exps)
    print('rho_hat ≈\n', np.round(rho_hat, 6))
    print('Fidelidade com |+>:', round(fidelity(rho_hat, rho_ideal), 6))

    # Exemplo com estado arbitrário via rotações de Euler
    theta, phi = 1.1, 0.3
    def prep_arbitrary(qc, qidx):
        qc.ry(theta, qidx)
        qc.rz(phi, qidx)
    rho_hat2, exps2, _ = shadow_tomography_rho(secret_prep=prep_arbitrary, shots_per_basis=6000, return_all=True)
    # Rho ideal correspondente
    v = np.array([np.cos(theta/2), np.exp(1j*phi)*np.sin(theta/2)], dtype=complex)
    rho_id2 = np.outer(v, v.conj())
    print('\nEstado arbitrário:')
    print('⟨X⟩,⟨Y⟩,⟨Z⟩:', exps2)
    print('Fidelidade com ideal:', round(fidelity(rho_hat2, rho_id2), 6))


⟨X⟩,⟨Y⟩,⟨Z⟩ estimados: {'X': 1.0, 'Y': -0.02, 'Z': -0.02}
rho_hat ≈
 [[0.49+0.j   0.5 +0.01j]
 [0.5 -0.01j 0.51+0.j  ]]
Fidelidade com |+>: 1.0

Estado arbitrário:
⟨X⟩,⟨Y⟩,⟨Z⟩: {'X': 0.8496666666666667, 'Y': 0.24566666666666667, 'Z': 0.453}
Fidelidade com ideal: 0.996794


In [16]:
# Shadow tomography para o qubit recebido em um circuito de teleporte quântico
# Agora com SWAPs para "afastar" Bob antes da tomografia.

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import numpy as np

# --- Utilidades de álgebra ---
I2 = np.eye(2, dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)

def bloch_to_rho(rx, ry, rz):
    return 0.5*(I2 + rx*X + ry*Y + rz*Z)

# --- Circuito de teleporte canônico com SWAPs ---

def teleport_circuit(secret_prep=None, n_swaps=2):
    """
    Teleporte quântico com correções condicionais e SWAPs para afastar Bob.

    q0: Secret (estado a ser teletransportado)
    q1: Alice
    q2: Bob (receptor final, será afastado por SWAPs)
    """
    q = QuantumRegister(3 + n_swaps, 'q')  # espaço extra para afastar Bob
    c0 = ClassicalRegister(1, 'c0')
    c1 = ClassicalRegister(1, 'c1')
    qc = QuantumCircuit(q, c0, c1, name='teleport')

    # Estado secreto |ψ> em q[0]
    if secret_prep is None:
        secret_prep = lambda qc_, qidx: qc_.h(qidx)
    secret_prep(qc, q[0])

    # Par EPR entre q1 (Alice) e q2 (Bob inicial)
    qc.h(q[1])
    qc.cx(q[1], q[2])

    # Medição de Bell em (q0,q1)
    qc.cx(q[0], q[1])
    qc.h(q[0])
    qc.measure(q[0], c0)
    qc.measure(q[1], c1)

    # Correções condicionais em q2 (Bob inicial)
    with qc.if_test((c1, 1)):
        qc.x(q[2])
    with qc.if_test((c0, 1)):
        qc.z(q[2])

    # SWAPs para "afastar" Bob até o último qubit
    bob_idx = 2
    for k in range(n_swaps):
        qc.swap(bob_idx + k, bob_idx + k + 1)
    final_bob = bob_idx + n_swaps

    return qc, final_bob

# --- Pauli measurements ---
_basis_rot = {
    'Z': [],
    'X': [('h', )],
    'Y': [('sdg',), ('h',)],
}

def _attach_pauli_measure_on_bob(base_circ: QuantumCircuit, bob_idx: int, basis: str):
    circ = base_circ.copy()
    cb = ClassicalRegister(1, f'cb_{basis}')
    circ.add_register(cb)
    for step in _basis_rot[basis]:
        gate = step[0]
        if gate == 'h':
            circ.h(bob_idx)
        elif gate == 'sdg':
            circ.sdg(bob_idx)
    circ.measure(bob_idx, cb)
    return circ

def estimate_pauli_expectations(sim, base_circ, bob_idx, shots_per_basis=2000, seed=1234):
    exps = {}
    for basis in ['X','Y','Z']:
        circ = _attach_pauli_measure_on_bob(base_circ, bob_idx, basis)
        res = sim.run(circ, shots=shots_per_basis, seed_simulator=seed).result()
        counts = res.get_counts()
        p0 = sum(v for k,v in counts.items() if k[0]=='0')
        p1 = sum(v for k,v in counts.items() if k[0]=='1')
        total = p0 + p1 if (p0+p1)>0 else 1
        exps[basis] = (p0 - p1)/total
    return exps

def shadow_tomography_rho(secret_prep=None, shots_per_basis=2000, n_swaps=2, return_all=False, seed=1234):
    base, bob_idx = teleport_circuit(secret_prep=secret_prep, n_swaps=n_swaps)
    sim = AerSimulator()
    exps = estimate_pauli_expectations(sim, base, bob_idx, shots_per_basis=shots_per_basis, seed=seed)
    rho_hat = bloch_to_rho(exps['X'], exps['Y'], exps['Z'])
    if return_all:
        return rho_hat, exps, base
    return rho_hat

# --- Exemplo ---
if __name__ == '__main__':
    rho_hat, exps, circ = shadow_tomography_rho(return_all=True, shots_per_basis=4000, n_swaps=3)
    print('⟨X⟩,⟨Y⟩,⟨Z⟩ estimados:', exps)
    print('rho_hat ≈\n', np.round(rho_hat, 6))


⟨X⟩,⟨Y⟩,⟨Z⟩ estimados: {'X': 1.0, 'Y': -0.02, 'Z': -0.02}
rho_hat ≈
 [[0.49+0.j   0.5 +0.01j]
 [0.5 -0.01j 0.51+0.j  ]]
